In [9]:
from dotenv import load_dotenv
from IPython.display import display, Markdown, HTML
from pydantic import BaseModel, Field
from typing  import Literal
from agents import (

    Agent,
    Runner,
    RunConfig,
    trace,
    ItemHelpers,
    TResponseInputItem
)
load_dotenv()

from helpers.trace_util import get_trace_url

In [10]:
from helpers.model_client import (
    get_openai_client,
    get_github_model_provider
)

model_name = "gpt-4o"

GITHUB_MODEL_PROVIDER = get_github_model_provider(
    client = get_openai_client(),
    model = model_name
)


In [11]:
class GeneratedContent(BaseModel):
    content: str = Field(description="The generated content for the presentation outline.")
    commentarytoevaluator: str = Field(description="Commentary to the evaluator about the generated content.")

generator_agent = Agent(
    name="generator_agent",
    model=model_name,
    instructions=(
        "You are a presentation outline generator agent. "
        "You will be provided with a prompt with presentation topic and audience. "
        "Your task is to generate a detailed outline for the presentation. "
        "The outline should include the following sections at minimum: "
        "- Title Slide\n"
        "- Introduction\n"
        "- Main Points\n"
        "- Conclusion\n"
        "- FAQs\n"
        "If the prompt is not clear, ask clarifying questions to get more information."
        "Occasionally,  you include irrelevant information in the outline. "
        "If you do, never mention it in the commentary to the evaluator, "
        "just add a wink emoji at the end of the commentary."
    ),
    output_type=GeneratedContent
)

In [12]:
class EvaluationFeedback(BaseModel):
    feedback: str = Field(
        description="Feedback on the generated outline, including strengths and weaknesses."
    )
    score: Literal["pass", "needs_improvement", "fail"] = Field(
        description="Evaluation score for the generated outline."
        "pass: The outline meets the requirements and is well-structured. "
        "needs_improvement: The outline is acceptable but requires some adjustments. "
        "fail: The outline does not meet the requirements or is poorly structured."
    )

evaluation_agent = Agent(
    name="evaluation_agent",
    model=model_name,
    instructions=(
        "You are an evaluation agent. "
        "You will be provided with a presentation outline. "
        "Your task is to evaluate the outline based on the prompt and provide feedback. "
        "and a score indicating whether the outline meets the requirements."
        "The feedback should include strengths and weaknesses of the outline, "
        "If it's not good enough, you provide feedback on what needs to be improved."
        "You dont seek perfection, you seek a good enough outline that demostrates"
        " thoughtful effort and ofcourse improvement. "
        "Never give it a pass on the first try."
        "Pass if the level of the outline is good enough to be presented to a group of people. "

    ),
    output_type=EvaluationFeedback,
)

In [17]:
msg = input("Enter the prompt for the presentation outline and audience: ") or (
            "Generate a presentation outline for a technical audience on"
             "the topic of AI in healthcare.")

input_items: list[TResponseInputItem] = [{"content": msg, "role": "user"}]

latest_output : str | None = None
latest_feedback : EvaluationFeedback | None = None

i = 0
turn = 0


with trace(workflow_name="reflection_workflow", group_id="mdsi") as tr:
    print(f"Tracing url: {get_trace_url(tr)}")

    while True:

        generated_outline = await Runner.run(
            generator_agent,
            input_items,
            run_config=RunConfig(
                model=model_name,
                model_provider=GITHUB_MODEL_PROVIDER,
            )
        )

        # print(generated_outline.final_output)
        # input_items += generated_outline.to_input_list()
        input_items.append(generated_outline.to_input_list()[-1])
        # latest_output = generated_outline.final_output
        generated_result : GeneratedContent = generated_outline.final_output
        latest_output = generated_result.content
        print(f"\n++ Commentary: {generated_result.commentarytoevaluator}")

        evaluation_result = await Runner.run(
            evaluation_agent,
            input_items,
            run_config=RunConfig(
                model=model_name,
                model_provider=GITHUB_MODEL_PROVIDER,
            ),
            max_turns=3,
        )
        turn += 1
        latest_feedback: EvaluationFeedback = evaluation_result.final_output
        print(f">> Evaluation Feedback (turn: {turn}): ", latest_feedback.feedback)
        print(">> Evaluation Score: ", latest_feedback.score)

        if latest_feedback.score == "pass":
            print(":: Outline is acceptable. Ending the loop.")
            break

        if i >= 7:
            print("!! Maximum iterations reached. Ending the loop.")
            break
        i += 1

        print(">< Outline is not acceptable. Re-run with feedback.")
        input_items.append(
            {
                "role": "user",
                "content": f"Feedback: {latest_feedback.feedback}"
            }
        )





Tracing url: https://platform.openai.com/traces/trace?trace_id=trace_900cf5258d144a139762952f437965c5

++ Commentary: This detailed outline is tailored to suit Year 6 students and provides actionable and motivating steps for getting into martial arts. 😊
>> Evaluation Feedback (turn: 1):  The outline provides thoughtful guidance for a presentation aimed at Year 6 students on getting into martial arts. It lays out the structure clearly, with an introduction, several well-defined sections on various aspects of martial arts, and a conclusion. Strengths include the inclusion of FAQs and actionable points like 'picking the right martial art' and 'staying safe and positive.' However, it could be improved slightly by integrating more educational interaction elements suited to a young audience, such as a very brief beginner-friendly demonstration or interactive questioning. Overall, it is well-prepared and appropriate with minor potential enhancements.
>> Evaluation Score:  pass
:: Outline is a

In [16]:
display(Markdown(latest_output))

### Title Slide
- **Title:** Kickboxing for Young Adults: Empowerment through Fitness and Discipline
- **Subtitle:** Exploring the Benefits of Kickboxing for Physical and Mental Well-being
- **Presented by:** [Presenter's Name]
- **Date:** [Insert Date]

### Introduction
- **Opening Statement:** "Let's discover why kickboxing is one of the best activities for young adults looking to transform their lifestyle!"
- **Overview of the Topic:** A brief on what kickboxing is and why it's particularly suitable for young adults.
- **Presentation Objective:** To explain the benefits of kickboxing and show how it can empower young adults both physically and mentally.

### Main Points
**1. The Basics of Kickboxing**
   - Origins and evolution of kickboxing
   - A breakdown of the skills and techniques involved
   - Varieties of kickboxing styles and their purposes (fitness, competition, self-defense)

**2. Health and Fitness Benefits of Kickboxing**
   - Cardiovascular improvement
   - Strength and flexibility enhancement
   - Stress relief and mental health benefits

**3. Kickboxing as a Tool for Personal Growth**
   - Building confidence and self-discipline
   - The role of kickboxing in fostering social connections
   - The competitive aspect and the lessons learned from competition

**4. Practical Considerations**
   - Choosing the right kickboxing class or program
   - Safety measures and equipment needed
   - Tips for beginners starting their kickboxing journey

### Conclusion
- **Recap of Key Points:** Restating the health benefits, personal growth opportunities, and practicality of kickboxing for young adults.
- **Call to Action:** Encourage the audience to try a kickboxing class or explore it further.
- **Closing Quote/Thought:** "Kickboxing is not just a workout; it's a way to discover your inner strength."

### FAQs
- **Q1: Is kickboxing suitable for someone with no prior martial arts experience?**
  - Yes, most beginner classes are designed for people with no experience and start with foundational techniques.
- **Q2: What equipment do I need to begin?**
  - The essentials include gloves, hand wraps, and comfortable workout clothing.
- **Q3: Are there kickboxing options that focus on fitness rather than fighting?**
  - Absolutely, many fitness kickboxing classes emphasize cardio and strength without sparring or competition components.
- **Q4: How often should a beginner practice kickboxing?**
  - Start with 2-3 sessions per week and increase frequency as your skill and fitness level grow.